In [55]:
import requests
from bs4 import BeautifulSoup
import numpy as np

In [14]:
# url com o bairro agua verde
url = 'https://www.vivareal.com.br/aluguel/parana/curitiba/bairros/agua-verde/apartamento_residencial/#onde=BR-Parana-NULL-Curitiba-Barrios-Agua_Verde&tipos=apartamento_residencial'
page_content = requests.get(url).content

In [15]:
page_content

b' <!DOCTYPE html> <html lang="pt-BR"> <head> <script> var pageCategory = \'RESULT PAGE\';\n\n  const clickstreamTracker = function (rawData) {\n    const clickstreamTrackerEndpoint = \'https://tracking.vivareal.com/events/v2\';\n    const data = JSON.stringify(rawData);\n\n    if (navigator.sendBeacon) {\n      navigator.sendBeacon(clickstreamTrackerEndpoint, data);\n    } else {\n      const ajax = new XMLHttpRequest();\n      ajax.open(\'POST\', clickstreamTrackerEndpoint, true);\n      ajax.send(data);\n    }\n  };\n\n  (function () {\n    const getCookie = function(name) {\n      const value = `; ${document.cookie}`;\n      const parts = value.split(`; ${name}=`);\n\n      if (parts.length == 2) return parts.pop().split(\';\').shift();\n      return \'\';\n    }\n\n    const getExperimentId = function() {\n      const gaExp = getCookie(\'_gaexp\');\n\n      if (gaExp) {\n        return gaExp.split(\'!\')\n          .map(v => v.replace(/GAX([0-9]+\\.){2}/, \'\').split(\'.\')[0])\n 

In [16]:
f = open('vivarealtest.html', 'x')

In [17]:
f.write(str(page_content))
f.close()

In [18]:
soup = BeautifulSoup(page_content, 'html.parser')

In [21]:
cards = soup.find_all('div', 'property-card__main-content')

In [24]:
len(cards)

36

In [63]:
cards_info = []
for card in cards:
    titulo = card.find('a', 'property-card__title js-cardLink js-card-title').text.strip()
    link_detalhes = card.find('a', 'property-card__title js-cardLink js-card-title')['href']
    tamanho = card.find('span', 'property-card__detail-value js-property-card-value property-card__detail-area js-property-card-detail-area').text.strip()
    quartos = card.find('li', 'property-card__detail-item property-card__detail-room js-property-detail-rooms').find('span', 'property-card__detail-value js-property-card-value').text.strip()
    suites = card.find('li', 'property-card__detail-item property-card__detail-item-extra js-property-detail-suites').find('span', 'property-card__detail-value js-property-card-value').text.strip()
    banheiros = card.find('li', 'property-card__detail-item property-card__detail-bathroom js-property-detail-bathroom').find('span', 'property-card__detail-value js-property-card-value').text.strip()
    vagas_carro = card.find('li', 'property-card__detail-item property-card__detail-garage js-property-detail-garages').find('span', 'property-card__detail-value js-property-card-value').text.strip()
    valor_aluguel = card.find('div', 'property-card__price js-property-card-prices js-property-card__price-small').text.strip().split()[1].replace('.', '')
    valor_condominio = card.find('div', 'property-card__price-details--condo')
    if valor_condominio:
        valor_condominio = valor_condominio.find('strong', 'js-condo-price').text.strip().split()[1].replace('.', '')
    else:
        valor_condominio = None
    cards_info.append({'titulo': titulo, 'link_detalhes': link_detalhes, 'tamanho': tamanho, 'quartos': quartos,
                      'suites': suites, 'banheiros': banheiros, 'vagas_carro': vagas_carro, 
                       'valor_aluguel': valor_aluguel, 'valor_condominio': valor_condominio})

In [64]:
cards_info

[{'titulo': 'Apartamento com 3 Quartos para Aluguel, 173m²',
  'link_detalhes': '/imovel/apartamento-3-quartos-agua-verde-bairros-curitiba-com-garagem-173m2-aluguel-RS3000-id-2490462708/',
  'tamanho': '173',
  'quartos': '3',
  'suites': '1',
  'banheiros': '3',
  'vagas_carro': '2',
  'valor_aluguel': '3000',
  'valor_condominio': '700'},
 {'titulo': 'Apartamento com 2 Quartos para Aluguel, 99m²',
  'link_detalhes': '/imovel/apartamento-2-quartos-agua-verde-bairros-curitiba-com-garagem-99m2-aluguel-RS2300-id-2491073478/',
  'tamanho': '99',
  'quartos': '2',
  'suites': '1',
  'banheiros': '3',
  'vagas_carro': '1',
  'valor_aluguel': '2300',
  'valor_condominio': '650'},
 {'titulo': 'Apartamento com 4 Quartos para Aluguel, 350m²',
  'link_detalhes': '/imovel/apartamento-4-quartos-agua-verde-bairros-curitiba-com-garagem-350m2-aluguel-RS6700-id-2493130017/',
  'tamanho': '350',
  'quartos': '4',
  'suites': '4',
  'banheiros': '5',
  'vagas_carro': '4',
  'valor_aluguel': '6700',
  'v